In [ ]:
!pip install -U transformers==4.43.3 accelerate==0.33.0 peft==0.11.1 trl==0.9.6 datasets==2.20.0 pandas==2.2.2
!pip install -U bitsandbytes==0.43.1
!pip install -U triton==2.3.0


# Klue -Roberta - small

In [ ]:
# -*- coding: utf-8 -*-
import os, random
import numpy as np
import pandas as pd
import torch

from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    set_seed,
)
from google.colab import drive
drive.mount('/content/drive')

# ===============================
# 0) 시드 고정
# ===============================
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] device: {device}")

# ===============================
# 1) 데이터 로딩
# ===============================
CSV_PATH = r"/content/drive/MyDrive/KDH/dataset/llm_datasets_0817_non_utf8.csv"
df = pd.read_csv(CSV_PATH)

df = df.dropna(subset=["label", "text"])
df = df[df["label"].isin([0, 1])]
df = df.reset_index(drop=True)
df["label"] = df["label"].astype(int)
df["text"]  = df["text"].astype(str)

print("[INFO] Dataset size:", len(df))
print(df["label"].value_counts())

# ===============================
# 2) 데이터 분할
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"],
    test_size=0.2,
    stratify=df["label"],
    random_state=SEED
)

# ===============================
# 3) Tokenizer & Dataset
# ===============================
MODEL_NAME = "klue/roberta-small"   # ✅ Roberta-small 사용
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.model_max_length = 512

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True)

train_ds = Dataset.from_dict({"text": X_train.tolist(), "labels": y_train.tolist()})
test_ds  = Dataset.from_dict({"text": X_test.tolist(),  "labels": y_test.tolist()})

train_ds = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
test_ds  = test_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ===============================
# 4) 모델 로딩
# ===============================
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
).to(device)

# ===============================
# 5) 학습 설정
# ===============================
training_args = TrainingArguments(
    output_dir="./roberta-small-classifier",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    warmup_ratio=0.1,
    report_to="none"
)

from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# ===============================
# 6) 학습
# ===============================
trainer.train()

# ===============================
# 7) 예측 후 Confusion Matrix 출력
# ===============================
out = trainer.predict(test_ds)
logits = out.predictions
probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
y_true = out.label_ids
y_pred = probs.argmax(axis=-1)

cm = confusion_matrix(y_true, y_pred)
print("=== Confusion Matrix ===")
print(cm)

# ============================
# 추론 속도 + 파라미터 수 + 메모리 사용량
# -*- coding: utf-8 -*-
import os, random, time, psutil
import numpy as np
import pandas as pd
import torch

from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    set_seed,
)
from google.colab import drive
drive.mount('/content/drive')

# ===============================
# 0) 시드 고정
# ===============================
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] device: {device}")

# ===============================
# 1) 데이터 로딩
# ===============================
CSV_PATH = r"/content/drive/MyDrive/KDH/dataset/llm_datasets_0817_non_utf8.csv"
df = pd.read_csv(CSV_PATH)

df = df.dropna(subset=["label", "text"])
df = df[df["label"].isin([0, 1])]
df = df.reset_index(drop=True)
df["label"] = df["label"].astype(int)
df["text"]  = df["text"].astype(str)

print("[INFO] Dataset size:", len(df))
print(df["label"].value_counts())

# ===============================
# 2) 데이터 분할
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"],
    test_size=0.2,
    stratify=df["label"],
    random_state=SEED
)

# ===============================
# 3) Tokenizer & Dataset
# ===============================
MODEL_NAME = "klue/roberta-small"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.model_max_length = 512

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True)

train_ds = Dataset.from_dict({"text": X_train.tolist(), "labels": y_train.tolist()})
test_ds  = Dataset.from_dict({"text": X_test.tolist(),  "labels": y_test.tolist()})

train_ds = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
test_ds  = test_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ===============================
# 4) 모델 로딩
# ===============================
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
).to(device)

# ===============================
# 5) 학습 설정
# ===============================
training_args = TrainingArguments(
    output_dir="./roberta-small-classifier",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    warmup_ratio=0.1,
    report_to="none"
)

from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# ===============================
# 6) 학습
# ===============================
trainer.train()

# ===============================
# 7) 예측 후 Confusion Matrix 출력
# ===============================
out = trainer.predict(test_ds)
logits = out.predictions
probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
y_true = out.label_ids
y_pred = probs.argmax(axis=-1)

cm = confusion_matrix(y_true, y_pred)
print("=== Confusion Matrix ===")
print(cm)

# ===============================
# 8) 추가 성능/효율 지표
# ===============================
# ===============================
# 8) 추가 성능/효율 지표  (교체본)
# ===============================
from torch.utils.data import DataLoader

# (1) 추론 속도 측정 — DataLoader + data_collator 사용(패딩/타입 안전)
eval_loader = DataLoader(test_ds, batch_size=32, collate_fn=data_collator)

batch = next(iter(eval_loader))              # {'input_ids': [B,L], 'attention_mask': [B,L], 'labels': [B]}
batch.pop("labels", None)                    # 라벨 제거
batch = {k: v.to(device) for k, v in batch.items()}  # 디바이스 이동

model.eval()
with torch.no_grad():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.reset_peak_memory_stats()  # ⬅️ 추론 피크메모리만 보려면 리셋
    t0 = time.time()
    _ = model(**batch)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    t1 = time.time()

B = batch["input_ids"].size(0)
latency = (t1 - t0) / B
throughput = B / (t1 - t0)

print(f"\n=== 추론 속도 ===")
print(f"평균 Latency: {latency*1000:.2f} ms/샘플")
print(f"Throughput : {throughput:.2f} 샘플/초")

# (2) 파라미터 수
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n=== 파라미터 수 ===")
print(f"전체 파라미터 수: {total_params/1e6:.2f} M")
print(f"학습가능 파라미터 수: {trainable_params/1e6:.2f} M")

# (3) 메모리 사용량
print(f"\n=== 메모리 사용량 ===")
if torch.cuda.is_available():
    allocated = torch.cuda.max_memory_allocated(device) / (1024**2)
    reserved  = torch.cuda.max_memory_reserved(device)  / (1024**2)
    print(f"GPU 메모리 (할당): {allocated:.2f} MB")
    print(f"GPU 메모리 (예약): {reserved:.2f} MB")
else:
    print(f"CPU 메모리 사용량: {psutil.Process(os.getpid()).memory_info().rss/(1024**2):.2f} MB")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] device: cuda
[INFO] Dataset size: 1033
label
1    702
0    331
Name: count, dtype: int64


Map:   0%|          | 0/826 [00:00<?, ? examples/s]

Map:   0%|          | 0/207 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.235300,0.182160,0.917874,0.939502,0.942857,0.936170
2,0.235700,0.250313,0.937198,0.953737,0.957143,0.950355
3,0.089800,0.149990,0.961353,0.971831,0.965035,0.978723
4,0.015300,0.299058,0.937198,0.953737,0.957143,0.950355
5,0.000500,0.308642,0.942029,0.957143,0.964029,0.950355


...

Map:   0%|          | 0/826 [00:00<?, ? examples/s]

Map:   0%|          | 0/207 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.235300,0.182160,0.917874,0.939502,0.942857,0.936170
2,0.235700,0.250313,0.937198,0.953737,0.957143,0.950355
3,0.089800,0.149990,0.961353,0.971831,0.965035,0.978723
4,0.015300,0.299058,0.937198,0.953737,0.957143,0.950355
5,0.000500,0.308642,0.942029,0.957143,0.964029,0.950355


=== Confusion Matrix ===
[[ 61   5]
 [  3 138]]

=== 추론 속도 ===
평균 Latency: 0.40 ms/샘플
Throughput : 2499.26 샘플/초

=== 파라미터 수 ===
전체 파라미터 수: 68.09 M
학습가능 파라미터 수: 68.09 M

=== 메모리 사용량 ===
GPU 메모리 (할당): 1152.30 MB
GPU 메모리 (예약): 1874.00 MB


# Klue-Roberta-small + base - Knowledge Distillation

In [ ]:
# -*- coding: utf-8 -*-
import os, random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
)
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    set_seed,
)
from google.colab import drive
drive.mount('/content/drive')

# ===============================
# 0) 시드 고정
# ===============================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] device: {device}")

# ===============================
# 1) 데이터 로딩
# ===============================
CSV_PATH = r"/content/drive/MyDrive/KDH/dataset/llm_datasets_0817_non_utf8.csv"
df = pd.read_csv(CSV_PATH)

df = df.dropna(subset=["label", "text"]).copy()
df = df[df["label"].isin([0, 1])].copy()
df["label"] = df["label"].astype(np.int64)   # torch.long

df["text"]  = df["text"].astype(str)
df = df.reset_index(drop=True)

print("[INFO] unique labels:", sorted(df["label"].unique()))

# ===============================
# 2) 데이터 분할
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"],
    test_size=0.2,
    stratify=df["label"],
    random_state=SEED
)

train_ds = Dataset.from_dict({
    "text": X_train.tolist(),
    "labels": y_train.astype(np.int64).tolist()
})

test_ds  = Dataset.from_dict({"text": X_test.tolist(),
                              "labels": y_test.astype(np.int64).tolist()})

# ===============================
# 3) Tokenizer (학생 기준)
# ===============================
teacher_model_name = "klue/roberta-base"
student_model_name = "klue/roberta-small"

# ✅ Roberta tokenizer 하나만 사용
tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_enc = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
test_enc  = test_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

train_enc.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_enc.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# ✅ collator도 같은 tokenizer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# ===============================
# 4) 모델 로딩
# ===============================
teacher = AutoModelForSequenceClassification.from_pretrained(
    teacher_model_name, num_labels=2
).to(device)

student = AutoModelForSequenceClassification.from_pretrained(
    student_model_name, num_labels=2
).to(device)

teacher.eval()  # 교사는 고정(freeze)

# ===============================
# 5) Distillation Trainer 정의
# ===============================
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.5, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher = teacher_model.eval()
        self.temperature = temperature
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs["labels"].view(-1).to(torch.long)


        # 라벨 shape/dtype 보정
        labels = labels.view(-1).to(torch.long)

        student_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs_student = model(**student_inputs)
        student_logits = (
            outputs_student.logits if hasattr(outputs_student, "logits") else outputs_student[0]
        )
        loss_ce = self.ce_loss(student_logits, labels)

        # 👉 교사 입력 만들기
        teacher_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        with torch.no_grad():
            outputs_teacher = self.teacher(**teacher_inputs)
            teacher_logits = (
                outputs_teacher.logits if hasattr(outputs_teacher, "logits") else outputs_teacher[0]
            )


        # NaN/Inf guard
        if torch.isnan(student_logits).any() or torch.isnan(teacher_logits).any():
            raise FloatingPointError("NaN detected in logits")

        # KD loss
        T = self.temperature
        loss_kd = F.kl_div(
            F.log_softmax(student_logits / T, dim=-1),
            F.softmax(teacher_logits / T, dim=-1),
            reduction="batchmean"
        ) * (T * T)

        loss = self.alpha * loss_ce + (1 - self.alpha) * loss_kd
        return (loss, {"logits": student_logits}) if return_outputs else loss




# ===============================
# 6) 학습 설정
# ===============================
training_args = TrainingArguments(
    output_dir="./distilled-klue/roberta-small",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
    fp16=False,              # Colab에서 안정성 위해 끔
    warmup_ratio=0.1,
    report_to="none",
    save_safetensors=False   # ✅ contiguous 에러 방지
)

# ===============================
# 7) 평가지표
# ===============================
def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    labels = eval_pred.label_ids

    # ✅ predictions가 tuple일 때 처리
    if isinstance(logits, tuple):
        if len(logits) > 0:
            logits = logits[0]
        else:
            raise ValueError("logits is an empty tuple!")

    preds = logits.argmax(axis=-1)
    p, r, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", pos_label=1
    )
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}




# ===============================
# 8) Trainer 실행
# ===============================
trainer = DistillationTrainer(
    model=student,
    teacher_model=teacher,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=test_enc,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# ===============================
# 9) Confusion Matrix 출력
# ===============================
out = trainer.predict(test_enc)
logits = out.predictions
probs = torch.softmax(torch.from_numpy(logits), dim=1).numpy()
y_true = out.label_ids
y_pred = probs.argmax(axis=-1)

cm = confusion_matrix(y_true, y_pred)
print("=== Confusion Matrix ===")
print(cm)

print("=== Classification Report ===")
print(classification_report(y_true, y_pred, digits=4))

# ===============================
# 10) 추가 성능/효율 지표  (수정본)
# ===============================
import time, psutil
from torch.utils.data import DataLoader

# ✅ 반드시 토크나이즈된 평가셋 사용
eval_loader = DataLoader(test_enc, batch_size=32, collate_fn=data_collator)

# 배치 하나 가져오기
batch = next(iter(eval_loader))  # dict_keys(['input_ids','attention_mask','labels'])
# 라벨 제거 + 디바이스 이동
batch_no_label = {k: v.to(device) for k, v in batch.items() if k != "labels"}

# ✅ 변수명 일치: student 모델로 추론
student.eval()
with torch.no_grad():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.reset_peak_memory_stats()  # 피크 메모리 리셋
    t0 = time.time()
    _ = student(**batch_no_label)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    t1 = time.time()

B = batch_no_label["input_ids"].size(0)
latency = (t1 - t0) / B
throughput = B / (t1 - t0)

print(f"\n=== 추론 속도 ===")
print(f"평균 Latency: {latency*1000:.2f} ms/샘플")
print(f"Throughput : {throughput:.2f} 샘플/초")

# (2) 파라미터 수 — student 기준
total_params = sum(p.numel() for p in student.parameters())
trainable_params = sum(p.numel() for p in student.parameters() if p.requires_grad)
print(f"\n=== 파라미터 수 ===")
print(f"전체 파라미터 수: {total_params/1e6:.2f} M")
print(f"학습가능 파라미터 수: {trainable_params/1e6:.2f} M")

# (3) 메모리 사용량
print(f"\n=== 메모리 사용량 ===")
if torch.cuda.is_available():
    allocated = torch.cuda.max_memory_allocated(device) / (1024**2)
    reserved  = torch.cuda.max_memory_reserved(device)  / (1024**2)
    print(f"GPU 메모리 (할당): {allocated:.2f} MB")
    print(f"GPU 메모리 (예약): {reserved:.2f} MB")
else:
    print(f"CPU 메모리 사용량: {psutil.Process(os.getpid()).memory_info().rss/(1024**2):.2f} MB")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] device: cuda
[INFO] unique labels: [0, 1]


Map:   0%|          | 0/826 [00:00<?, ? examples/s]

Map:   0%|          | 0/207 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.245622,0.927536,0.925676,0.971631,0.948097
2,No log,0.233949,0.937198,0.950704,0.957447,0.954064
3,No log,0.230061,0.961353,0.958621,0.985816,0.972028
4,No log,0.234607,0.932367,0.937931,0.964539,0.951049
5,No log,0.233800,0.946860,0.985075,0.936170,0.960000


=== Confusion Matrix ===
[[ 60   6]
 [  2 139]]
=== Classification Report ===
              precision    recall  f1-score   support

           0     0.9677    0.9091    0.9375        66
           1     0.9586    0.9858    0.9720       141

    accuracy                         0.9614       207
   macro avg     0.9632    0.9475    0.9548       207
weighted avg     0.9615    0.9614    0.9610       207


=== 추론 속도 ===
평균 Latency: 0.28 ms/샘플
Throughput : 3578.28 샘플/초

=== 파라미터 수 ===
전체 파라미터 수: 68.09 M
학습가능 파라미터 수: 68.09 M

=== 메모리 사용량 ===
GPU 메모리 (할당): 1698.07 MB
GPU 메모리 (예약): 2400.00 MB


# torch Pruning

In [ ]:
# -*- coding: utf-8 -*-
import os, random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.prune as prune

from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
)
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    set_seed,
)
from google.colab import drive
drive.mount('/content/drive')

# ===============================
# 0) 시드 고정
# ===============================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] device: {device}")

# ===============================
# 1) 데이터 로딩
# ===============================
CSV_PATH = r"/content/drive/MyDrive/KDH/dataset/llm_datasets_0817_non_utf8.csv"
df = pd.read_csv(CSV_PATH)

df = df.dropna(subset=["label", "text"]).copy()
df = df[df["label"].isin([0, 1])].copy()
df["label"] = df["label"].astype(np.int64)   # torch.long
df["text"]  = df["text"].astype(str)
df = df.reset_index(drop=True)

print("[INFO] unique labels:", sorted(df["label"].unique()))

# ===============================
# 2) 데이터 분할
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"],
    test_size=0.2,
    stratify=df["label"],
    random_state=SEED
)

train_ds = Dataset.from_dict({
    "text": X_train.tolist(),
    "labels": y_train.astype(np.int64).tolist()
})

test_ds  = Dataset.from_dict({
    "text": X_test.tolist(),
    "labels": y_test.astype(np.int64).tolist()
})

# ===============================
# 3) Tokenizer
# ===============================
teacher_model_name = "klue/roberta-base"
student_model_name = "klue/roberta-small"

tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_enc = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
test_enc  = test_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

train_enc.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_enc.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ===============================
# 4) 모델 로딩
# ===============================
teacher = AutoModelForSequenceClassification.from_pretrained(
    teacher_model_name, num_labels=2
).to(device)

student = AutoModelForSequenceClassification.from_pretrained(
    student_model_name, num_labels=2
).to(device)

teacher.eval()

# ===============================
# 5) Distillation Trainer
# ===============================
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.5, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher = teacher_model.eval()
        self.temperature = temperature
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs["labels"].view(-1).to(torch.long)

        student_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs_student = model(**student_inputs)

        # ✅ logits 안전 처리
        student_logits = outputs_student.logits if hasattr(outputs_student, "logits") else outputs_student[0]

        loss_ce = self.ce_loss(student_logits, labels)

        # teacher logits
        with torch.no_grad():
            outputs_teacher = self.teacher(**student_inputs)
            teacher_logits = outputs_teacher.logits if hasattr(outputs_teacher, "logits") else outputs_teacher[0]

        # KD loss
        T = self.temperature
        loss_kd = F.kl_div(
            F.log_softmax(student_logits / T, dim=-1),
            F.softmax(teacher_logits / T, dim=-1),
            reduction="batchmean"
        ) * (T * T)

        loss = self.alpha * loss_ce + (1 - self.alpha) * loss_kd
        return (loss, {"logits": student_logits}) if return_outputs else loss

# ===============================
# 6) 학습 설정
# ===============================
training_args = TrainingArguments(
    output_dir="./distilled-klue/roberta-small",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
    fp16=False,
    warmup_ratio=0.1,
    report_to="none",
    save_safetensors=False
)

# ===============================
# 7) 평가지표
# ===============================
def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    labels = eval_pred.label_ids

    if isinstance(logits, tuple):
        logits = logits[0]

    preds = logits.argmax(axis=-1)
    p, r, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", pos_label=1
    )
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

# ===============================
# 8) KD 학습
# ===============================
trainer = DistillationTrainer(
    model=student,
    teacher_model=teacher,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=test_enc,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# ===============================
# 9) Pruning 적용
# ===============================
print("\n[INFO] Applying pruning ...")

for name, module in student.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name="weight", amount=0.5)

# Sparsity 확인
for name, module in student.named_modules():
    if isinstance(module, torch.nn.Linear) and hasattr(module, "weight_mask"):
        sparsity = 100. * float(torch.sum(module.weight == 0)) / module.weight.nelement()
        print(f"Layer {name} | Sparsity: {sparsity:.2f}%")

# (선택) mask 병합
for name, module in student.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.remove(module, "weight")

# ===============================
# 10) 평가
# ===============================
out = trainer.predict(test_enc)
logits = out.predictions
probs = torch.softmax(torch.from_numpy(logits), dim=1).numpy()
y_true = out.label_ids
y_pred = probs.argmax(axis=-1)

cm = confusion_matrix(y_true, y_pred)
print("=== Confusion Matrix ===")
print(cm)

print("=== Classification Report ===")
print(classification_report(y_true, y_pred, digits=4))

# ===============================
# 11) 저장
# ===============================
student.save_pretrained("./distilled-roberta-pruned")
tokenizer.save_pretrained("./distilled-roberta-pruned")

# 10) 추가 성능/효율 지표  (수정본)
# ===============================
import time, psutil
from torch.utils.data import DataLoader

# ✅ 반드시 토크나이즈된 평가셋 사용
eval_loader = DataLoader(test_enc, batch_size=32, collate_fn=data_collator)

# 배치 하나 가져오기
batch = next(iter(eval_loader))  # dict_keys(['input_ids','attention_mask','labels'])
# 라벨 제거 + 디바이스 이동
batch_no_label = {k: v.to(device) for k, v in batch.items() if k != "labels"}

# ✅ 변수명 일치: student 모델로 추론
student.eval()
with torch.no_grad():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.reset_peak_memory_stats()  # 피크 메모리 리셋
    t0 = time.time()
    _ = student(**batch_no_label)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    t1 = time.time()

B = batch_no_label["input_ids"].size(0)
latency = (t1 - t0) / B
throughput = B / (t1 - t0)

print(f"\n=== 추론 속도 ===")
print(f"평균 Latency: {latency*1000:.2f} ms/샘플")
print(f"Throughput : {throughput:.2f} 샘플/초")

# (2) 파라미터 수 — student 기준
total_params = sum(p.numel() for p in student.parameters())
trainable_params = sum(p.numel() for p in student.parameters() if p.requires_grad)
print(f"\n=== 파라미터 수 ===")
print(f"전체 파라미터 수: {total_params/1e6:.2f} M")
print(f"학습가능 파라미터 수: {trainable_params/1e6:.2f} M")

# (3) 메모리 사용량
print(f"\n=== 메모리 사용량 ===")
if torch.cuda.is_available():
    allocated = torch.cuda.max_memory_allocated(device) / (1024**2)
    reserved  = torch.cuda.max_memory_reserved(device)  / (1024**2)
    print(f"GPU 메모리 (할당): {allocated:.2f} MB")
    print(f"GPU 메모리 (예약): {reserved:.2f} MB")
else:
    print(f"CPU 메모리 사용량: {psutil.Process(os.getpid()).memory_info().rss/(1024**2):.2f} MB")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] device: cuda
[INFO] unique labels: [0, 1]


Map:   0%|          | 0/826 [00:00<?, ? examples/s]

Map:   0%|          | 0/207 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.239210,0.951691,0.939597,0.992908,0.965517
2,No log,0.229788,0.956522,0.945946,0.992908,0.968858
3,No log,0.229716,0.961353,0.971631,0.971631,0.971631



[INFO] Applying pruning ...
Layer roberta.encoder.layer.0.attention.self.query | Sparsity: 50.00%
Layer roberta.encoder.layer.0.attention.self.key | Sparsity: 50.00%
Layer roberta.encoder.layer.0.attention.self.value | Sparsity: 50.00%
Layer roberta.encoder.layer.0.attention.output.dense | Sparsity: 50.00%
Layer roberta.encoder.layer.0.intermediate.dense | Sparsity: 50.00%
Layer roberta.encoder.layer.0.output.dense | Sparsity: 50.00%
Layer roberta.encoder.layer.1.attention.self.query | Sparsity: 50.00%
Layer roberta.encoder.layer.1.attention.self.key | Sparsity: 50.00%
Layer roberta.encoder.layer.1.attention.self.value | Sparsity: 50.00%
Layer roberta.encoder.layer.1.attention.output.dense | Sparsity: 50.00%
Layer roberta.encoder.layer.1.intermediate.dense | Sparsity: 50.00%
Layer roberta.encoder.layer.1.output.dense | Sparsity: 50.00%
Layer roberta.encoder.layer.2.attention.self.query | Sparsity: 50.00%
Layer roberta.encoder.layer.2.attention.self.key | Sparsity: 50.00%
Layer roberta

=== Confusion Matrix ===
[[ 62   4]
 [  7 134]]
=== Classification Report ===
              precision    recall  f1-score   support

           0     0.8986    0.9394    0.9185        66
           1     0.9710    0.9504    0.9606       141

    accuracy                         0.9469       207
   macro avg     0.9348    0.9449    0.9395       207
weighted avg     0.9479    0.9469    0.9472       207


=== 추론 속도 ===
평균 Latency: 0.32 ms/샘플
Throughput : 3167.08 샘플/초

=== 파라미터 수 ===
전체 파라미터 수: 68.09 M
학습가능 파라미터 수: 68.09 M

=== 메모리 사용량 ===
GPU 메모리 (할당): 1866.37 MB
GPU 메모리 (예약): 2422.00 MB


# QLoRA + KD

In [ ]:
# -*- coding: utf-8 -*-
import os, random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
)

from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    set_seed,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# ===============================
# 0) 시드 고정
# ===============================
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] device: {device}")

# ===============================
# 1) 데이터 로딩
# ===============================
CSV_PATH = r"/home/piai/PoscoAIproject/dataset/llm_datasets_0817_non_utf8.csv"
df = pd.read_csv(CSV_PATH)

df = df.dropna(subset=["label", "text"]).copy()
df = df[df["label"].isin([0, 1])].copy()
df["label"] = df["label"].astype(np.int64)
df["text"]  = df["text"].astype(str)
df = df.reset_index(drop=True)
print("[INFO] unique labels:", sorted(df["label"].unique()))

# ===============================
# 2) 데이터 분할
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"],
    test_size=0.2,
    stratify=df["label"],
    random_state=SEED
)

train_ds = Dataset.from_dict({
    "text": X_train.tolist(),
    "labels": y_train.astype(np.int64).tolist()
})
test_ds  = Dataset.from_dict({
    "text": X_test.tolist(),
    "labels": y_test.astype(np.int64).tolist()
})

# ===============================
# 3) Tokenizer
# ===============================
teacher_model_name = "klue/roberta-base"
student_model_name = "klue/roberta-small"

tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_enc = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
test_enc  = test_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

train_enc.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_enc.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ===============================
# 4) 모델 로딩 (Teacher / Student QLoRA)
# ===============================
# Teacher (float32 유지)
teacher = AutoModelForSequenceClassification.from_pretrained(
    teacher_model_name, num_labels=2,
    torch_dtype=torch.float32,
    device_map="auto"
)
teacher.eval()

# Student (Base는 4bit, Head는 float)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 1) Base encoder만 4bit로 로드
base_model = AutoModel.from_pretrained(
    student_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# 2) Classifier head는 float로 로드
student = AutoModelForSequenceClassification.from_pretrained(
    student_model_name,
    quantization_config=None,   # classifier는 양자화 안 함
    num_labels=2,
    device_map="auto"
)

# 3) Base encoder 교체 + classifier는 float 보장
student.roberta = base_model
student.classifier = student.classifier.to(torch.float32)

# --- 핵심: 항상 dict 출력 강제 ---
if hasattr(teacher, "config"): teacher.config.return_dict = True
if hasattr(student, "config"): student.config.return_dict = True
if hasattr(base_model, "config"): base_model.config.return_dict = True

# 4) QLoRA 준비
student.gradient_checkpointing_enable()
student.config.use_cache = False
student = prepare_model_for_kbit_training(student)

# 5) LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "key", "value", "dense"],  # RoBERTa 구조 기준
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)
student = get_peft_model(student, lora_config)

# ===============================
# 5) Distillation Trainer 정의
# ===============================
def _get_logits(outputs):
    # ModelOutput(dict-유사) 또는 tuple 모두 안전 처리
    if hasattr(outputs, "logits"):
        return outputs.logits
    if isinstance(outputs, (tuple, list)):
        return outputs[0]
    raise TypeError("Model outputs do not contain logits.")

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.5, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher = teacher_model.eval()
        self.temperature = temperature
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs["labels"].view(-1).to(torch.long)

        # Student forward (dict 강제 + 안전 추출)
        student_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs_student = model(**student_inputs, return_dict=True)
        student_logits = _get_logits(outputs_student).to(torch.float32)
        loss_ce = self.ce_loss(student_logits, labels)

        # Teacher forward (dict 강제 + 안전 추출)
        teacher_device = next(self.teacher.parameters()).device
        teacher_inputs = {k: v.to(teacher_device) for k, v in student_inputs.items()}
        with torch.no_grad():
            outputs_teacher = self.teacher(**teacher_inputs, return_dict=True)
            teacher_logits = _get_logits(outputs_teacher).to(torch.float32)

        # KD loss
        T = self.temperature
        loss_kd = F.kl_div(
            F.log_softmax(student_logits / T, dim=-1),
            F.softmax(teacher_logits / T, dim=-1),
            reduction="batchmean"
        ) * (T * T)

        loss = self.alpha * loss_ce + (1 - self.alpha) * loss_kd
        return (loss, {"logits": student_logits}) if return_outputs else loss

# ===============================
# 6) 학습 설정
# ===============================
training_args = TrainingArguments(
    output_dir="./qlora-distilled-roberta-small",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # <- 보통 'f1'로 둡니다
    greater_is_better=True,
    save_total_limit=2,
    fp16=True,
    gradient_checkpointing=True,
    warmup_ratio=0.1,
    report_to="none",
    save_safetensors=False
)

# ===============================
# 7) 평가지표
# ===============================
def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    labels = eval_pred.label_ids
    if isinstance(logits, (tuple, list)):
        logits = logits[0]
    preds = logits.argmax(axis=-1)
    p, r, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", pos_label=1, zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

# ===============================
# 8) Trainer 실행
# ===============================
trainer = DistillationTrainer(
    model=student,
    teacher_model=teacher,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=test_enc,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 학습 끝난 뒤
trainer.model.save_pretrained("/home/piai/PoscoAIproject/qlora-distilled-roberta-small")
tokenizer.save_pretrained("/home/piai/PoscoAIproject/qlora-distilled-roberta-small")

# ===============================
# 9) Confusion Matrix 출력
# ===============================
out = trainer.predict(test_enc)
logits = out.predictions if not isinstance(out.predictions, (tuple, list)) else out.predictions[0]
probs = torch.softmax(torch.from_numpy(logits), dim=1).numpy()
y_true = out.label_ids
y_pred = probs.argmax(axis=-1)

cm = confusion_matrix(y_true, y_pred)
print("=== Confusion Matrix ===")
print(cm)
print("=== Classification Report ===")
print(classification_report(y_true, y_pred, digits=4))

# 10) 추가 성능/효율 지표  (수정본)
# ===============================
import time, psutil
from torch.utils.data import DataLoader

# ✅ 반드시 토크나이즈된 평가셋 사용
eval_loader = DataLoader(test_enc, batch_size=32, collate_fn=data_collator)

# 배치 하나 가져오기
batch = next(iter(eval_loader))  # dict_keys(['input_ids','attention_mask','labels'])
# 라벨 제거 + 디바이스 이동
batch_no_label = {k: v.to(device) for k, v in batch.items() if k != "labels"}

# ✅ 변수명 일치: student 모델로 추론
student.eval()
with torch.no_grad():
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.reset_peak_memory_stats()  # 피크 메모리 리셋
    t0 = time.time()
    _ = student(**batch_no_label)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    t1 = time.time()

B = batch_no_label["input_ids"].size(0)
latency = (t1 - t0) / B
throughput = B / (t1 - t0)

print(f"\n=== 추론 속도 ===")
print(f"평균 Latency: {latency*1000:.2f} ms/샘플")
print(f"Throughput : {throughput:.2f} 샘플/초")

# (2) 파라미터 수 — student 기준
total_params = sum(p.numel() for p in student.parameters())
trainable_params = sum(p.numel() for p in student.parameters() if p.requires_grad)
print(f"\n=== 파라미터 수 ===")
print(f"전체 파라미터 수: {total_params/1e6:.2f} M")
print(f"학습가능 파라미터 수: {trainable_params/1e6:.2f} M")

# (3) 메모리 사용량
print(f"\n=== 메모리 사용량 ===")
if torch.cuda.is_available():
    allocated = torch.cuda.max_memory_allocated(device) / (1024**2)
    reserved  = torch.cuda.max_memory_reserved(device)  / (1024**2)
    print(f"GPU 메모리 (할당): {allocated:.2f} MB")
    print(f"GPU 메모리 (예약): {reserved:.2f} MB")
else:
    print(f"CPU 메모리 사용량: {psutil.Process(os.getpid()).memory_info().rss/(1024**2):.2f} MB")



[INFO] device: cuda


FileNotFoundError: [Errno 2] No such file or directory: '/home/piai/PoscoAIproject/dataset/llm_datasets_0817_non_utf8.csv'